In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


In [2]:
#Folder with cloud points files, folder for outputs
input_path="./"
output_path="./"
#dataname="sample_w_normals.xyz"
dataname="bunny_w_N.csv"


In [3]:
#Load data
point_cloud= np.loadtxt(input_path+dataname,skiprows=1,delimiter=',')

In [4]:
#Convert the data into open3d coordinates
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,3:6])
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,0:3])

In [5]:
#Visualization of the point cloud
o3d.visualization.draw_geometries([pcd])

In [6]:
#Compute the distances to the nearest neighbor and the average
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [7]:
#Create the mesh
bpa_geom = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [8]:
#Second method for the mesh
poisson_geom, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)

In [9]:
#Export the mesh to a file within the given output folder

#This works with ply and stl
o3d.io.write_triangle_mesh(output_path+"bunny_bpa.stl", bpa_geom)

#This only works with ply
o3d.io.write_triangle_mesh(output_path+"bunny_poisson.ply", poisson_geom)

True

In [10]:
#Import a mesh from the file and visualize it
bpa_mesh = o3d.io.read_triangle_mesh(output_path+"bunny_bpa.stl")
o3d.visualization.draw_geometries([bpa_mesh])


In [11]:
poisson_mesh = o3d.io.read_triangle_mesh(output_path+"bunny_poisson.ply")
o3d.visualization.draw_geometries([poisson_mesh])

In [12]:
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = poisson_mesh.vertices
density_mesh.triangles = poisson_mesh.triangles
density_mesh.triangle_normals = poisson_mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

visualize densities


In [13]:
original = o3d.io.read_triangle_mesh(input_path+"bunny.stl")
original_pcd=pcd

In [14]:
# Calculate point cloud for reconstucted mesh
computed_pcd=o3d.geometry.PointCloud()
computed_pcd.points=poisson_mesh.vertices

In [15]:
original_pcd.paint_uniform_color([0,0,1])
computed_pcd.paint_uniform_color([0.5,0.5,0])
o3d.visualization.draw_geometries([original_pcd,computed_pcd])

In [16]:
computed_pcd.paint_uniform_color([0.5,0.5,0])
o3d.visualization.draw_geometries([computed_pcd])

In [17]:
computed_pcd

PointCloud with 11923 points.

In [18]:
# Compute the error 
distance=original_pcd.compute_point_cloud_distance(computed_pcd)
distance=np.asarray(distance)
np.average(distance)

#sembra avere errore maggiore poisson rispetto bpa 

0.32337789458013266

In [19]:
dists = computed_pcd.compute_point_cloud_distance(pcd)
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
pcd_without_chair = computed_pcd.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_without_chair],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [20]:
pcd_without_chair

PointCloud with 11923 points.

In [21]:
a=original.sample_points_uniformly(20000)
b=bpa_mesh.sample_points_uniformly(20000)

In [22]:
dist= b.compute_point_cloud_distance(a)
np.average(dist)



0.28391981749484535